In [2]:
import tensorflow as tf
import tflearn
from tflearn.data_utils import pad_sequences
import pickle, pandas as pd, re, numpy as np, ast

from sklearn.model_selection import train_test_split

In [3]:
score_vect_dicts = pickle.load(open("score_vectors_dict", "rb"))
df = pd.read_csv("datasets/LMSR_rev2vec.csv")
df.head()

,Language,Movie_ID,Score,rev_vec
0,en,-800777728,9,[ 5.10346368e-02 3.13695297e-02 3.0434969...
1,en,-800777728,10,[ 5.85338362e-02 3.14969495e-02 9.2768417...
2,en,-1018312192,8,[ 5.58815002e-02 3.47756743e-02 -4.6352325...
3,en,-1018312192,4,[ 0.05872388 0.03473265 -0.01488676 0.101489...
4,en,-1018312192,7,[ 3.05793583e-02 3.45815420e-02 -6.1754882...


In [4]:
def parse_np_array(array_string, as_nparray=True):
    pattern = r'''# Match (mandatory) whitespace between...
              (?<=\]) # ] and
              \s+
              (?= \[) # [, or
              |
              (?<=[^\[\]\s]) 
              \s+
              (?= [^\[\]\s]) # two non-bracket non-whitespace characters
           '''
    fixed_string = re.sub(pattern, ',', array_string, flags=re.VERBOSE)
    if as_nparray:
        return np.array(ast.literal_eval(fixed_string))
    return ast.literal_eval(fixed_string)

In [20]:
X_ = list()
Y_ = list()
y = list()
for rev in df.iterrows():
    try:
        score = rev[1][2]
        vec = parse_np_array(rev[1][-1])
        Y_.append(score_vect_dicts[score])
        y.append(score)
    except:
        continue
    X_.append(vec)
X = np.array(X_)
Y = np.array(Y_)

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_train = pad_sequences(X_train, maxlen=300, value=0.)
X_test = pad_sequences(X_test, maxlen=300, value=0.)

In [21]:
X_train2, X_test2, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train2 = pad_sequences(X_train2, maxlen=300, value=0.)
X_test2 = pad_sequences(X_test2, maxlen=300, value=0.)

In [22]:
X_train2.shape, X_test2.shape

((688, 300), (296, 300))

In [24]:
with tf.device("/gpu:0"):
    net = tflearn.input_data([None, 300])
    net = tflearn.embedding(net, input_dim=1000, output_dim=300)
    net = tflearn.lstm(net, 300, dropout=0.8)
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.001)
    
    model = tflearn.DNN(net, tensorboard_verbose=0)
    model.fit(X_train, Y_train, validation_set(X_test, Y_test), show_metric=True, patch_size=64)

InvalidArgumentError: Node 'init_19/NoOp': Unknown input node '^is_training/Assign'

In [ ]:
net = tflearn.input_data([None, 300])
net = tflearn.embedding(net, input_dim=1000, output_dim=300)
net = tflearn.lstm(net, 300, dropout=0.8)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001)

In [ ]:
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(X_train2, y_train, validation_set(X_test, y_test), show_metric=True, patch_size=64)